In [1]:
from core import langda_solve

In [2]:
model = """
# Try langda solve:
operation(X,Y,Z) :-
    langda(LLM:"X plus Y equals Z",FUP:"false").
"""
additional_input = { "config": { "configurable": { "thread_id": "42" } },"prefix": "","langda_ext": ""}

In [3]:
result = langda_solve('simple',model,'deepseek-chat',additional_input=additional_input)

print(result)

### =========== processing init_node =========== ###
{'C5731F2D': 'operation(X, Y, Z) :- Z is X + Y.'}
[{'C5731F2D': 'operation(X, Y, Z) :- Z is X + Y.'}]
processing _decide_next_init ...
No langda needs to be updated, process end now...
### =========== processing summary_node =========== ###
*** test_result: ***

Running problog_test_tool...
Error evaluating Problog model:
    toks = self.label_tokens(string, root_tokens)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zhenzhili/miniforge3/envs/langda/lib/python3.11/site-packages/problog/parser.py", line 1186, in label_tokens
    raise ParseError(string, "Expected binary operator", t.location)
problog.parser.ParseError: Expected binary operator at 1:2.
{'C5731F2D': 'operation(X, Y, Z) :- Z is X + Y.'}
*** Running_time: 0s, 0 rounds in total.
# Try langda solve:
operation(X,Y,Z) :-
 
 Z is X + Y.


In [ ]:

MIXMNIST_test = MIXMNIST(prefix="Arabic", subset="test")
def engine(input:Iterable[int | bool]):
    return 1 if (input[0] >= 80 and input[1] <= 20) else 0
test_set = operation(2, "test", "Arabic", engine, seed=42)
query = test_set.to_query(0)

prompt_from_expert1 = "(24.7136° N, 46.6753° E)"
thismodel = different_languange_test(load_dataset="Arabic", logic_type="Engine",
                             train=False, load_pretrained=True, load_rule=True, prompt_from_expert=prompt_from_expert1)
test_set._get_label

def get_fake_queries(test_set, dataset):
    query = test_set.to_query(0)
    sub = {}
    for placeholder, tensor_term in query.substitution.items():
        print(placeholder,tensor_term)
        inner_term = tensor_term.args[0]
        if hasattr(inner_term, 'functor') and len(inner_term.args) > 0:
            source_name = inner_term.functor
            index = inner_term.args[0]
            
            # 处理index，可能是Constant类型
            if hasattr(index, 'value'):
                index = index.value
            else:
                index = int(index)
            sub[placeholder] = f"image_{dataset[index][1]}"
            print(source_name , index, dataset[index][1])
    query.substitute(sub).query

In [4]:
print(result)

# Try langda solve:
operation(X,Y,Z) :-
 
 Z is X + Y.


In [1]:
"""
DeepProbLog逻辑代码end-to-end验证框架
通过多种策略验证逻辑规则的正确性和一致性
"""

import torch
import numpy as np
from typing import List, Dict, Tuple, Any, Optional
from collections import defaultdict
from deepproblog.model import Model
from deepproblog.query import Query
from deepproblog.dataset import Dataset
from problog.logic import Term, Constant, Var
import itertools

class LogicVerificationFramework:
    """DeepProbLog逻辑代码验证框架"""
    
    def __init__(self, model: Model):
        self.model = model
        self.verification_results = {}
        
    def verify_logic_completeness(self, test_cases: List[Dict]) -> Dict[str, Any]:
        """
        验证逻辑规则的完整性
        检查是否存在遗漏的逻辑分支或规则
        """
        results = {
            'coverage_rate': 0.0,
            'missing_cases': [],
            'covered_cases': [],
            'logic_gaps': []
        }
        
        covered_cases = 0
        total_cases = len(test_cases)
        
        for case in test_cases:
            query = case['query']
            expected = case.get('expected_logic_path', None)
            
            # 执行查询并分析逻辑路径
            result = self.model.solve([query])[0]
            
            if len(result.result) > 0:
                covered_cases += 1
                results['covered_cases'].append(case)
                
                # 验证逻辑推理路径是否符合预期
                if expected and not self._verify_logic_path(result, expected):
                    results['logic_gaps'].append({
                        'case': case,
                        'expected_path': expected,
                        'actual_result': result.result
                    })
            else:
                results['missing_cases'].append(case)
        
        results['coverage_rate'] = covered_cases / total_cases if total_cases > 0 else 0
        return results
    
    def verify_logic_consistency(self, rule_groups: List[List[str]]) -> Dict[str, Any]:
        """
        验证逻辑规则之间的一致性
        检查相互冲突或重复的规则
        """
        results = {
            'consistency_score': 1.0,
            'conflicts': [],
            'redundancies': [],
            'circular_dependencies': []
        }
        
        # 检查规则冲突
        conflicts = self._detect_rule_conflicts(rule_groups)
        results['conflicts'] = conflicts
        
        # 检查规则冗余
        redundancies = self._detect_rule_redundancies(rule_groups)
        results['redundancies'] = redundancies
        
        # 检查循环依赖
        circular_deps = self._detect_circular_dependencies(rule_groups)
        results['circular_dependencies'] = circular_deps
        
        # 计算一致性分数
        total_issues = len(conflicts) + len(redundancies) + len(circular_deps)
        total_rules = sum(len(group) for group in rule_groups)
        results['consistency_score'] = max(0, 1 - (total_issues / total_rules))
        
        return results
    
    def verify_counterfactual_reasoning(self, base_cases: List[Dict]) -> Dict[str, Any]:
        """
        通过反事实推理验证逻辑正确性
        修改输入条件，验证输出变化是否符合逻辑预期
        """
        results = {
            'counterfactual_accuracy': 0.0,
            'failed_cases': [],
            'unexpected_behaviors': []
        }
        
        correct_predictions = 0
        total_tests = 0
        
        for base_case in base_cases:
            # 生成反事实变体
            counterfactuals = self._generate_counterfactuals(base_case)
            
            for cf_case in counterfactuals:
                total_tests += 1
                
                # 执行原始查询和反事实查询
                base_result = self.model.solve([base_case['query']])[0]
                cf_result = self.model.solve([cf_case['query']])[0]
                
                # 验证变化是否符合预期
                expected_change = cf_case.get('expected_change', None)
                if expected_change:
                    actual_change = self._analyze_result_change(base_result, cf_result)
                    
                    if self._matches_expected_change(actual_change, expected_change):
                        correct_predictions += 1
                    else:
                        results['failed_cases'].append({
                            'base_case': base_case,
                            'counterfactual': cf_case,
                            'expected_change': expected_change,
                            'actual_change': actual_change
                        })
        
        results['counterfactual_accuracy'] = correct_predictions / total_tests if total_tests > 0 else 0
        return results
    
    def verify_boundary_conditions(self, predicates: List[str]) -> Dict[str, Any]:
        """
        验证逻辑规则在边界条件下的行为
        """
        results = {
            'boundary_robustness': 0.0,
            'edge_case_failures': [],
            'undefined_behaviors': []
        }
        
        total_boundary_tests = 0
        successful_tests = 0
        
        for predicate in predicates:
            # 生成边界条件测试用例
            boundary_cases = self._generate_boundary_cases(predicate)
            
            for case in boundary_cases:
                total_boundary_tests += 1
                
                try:
                    result = self.model.solve([case['query']])[0]
                    
                    if case.get('should_succeed', True):
                        if len(result.result) > 0:
                            successful_tests += 1
                        else:
                            results['edge_case_failures'].append({
                                'case': case,
                                'expected': 'success',
                                'actual': 'failure'
                            })
                    else:
                        if len(result.result) == 0:
                            successful_tests += 1
                        else:
                            results['edge_case_failures'].append({
                                'case': case,
                                'expected': 'failure',
                                'actual': 'success'
                            })
                            
                except Exception as e:
                    results['undefined_behaviors'].append({
                        'case': case,
                        'error': str(e)
                    })
        
        results['boundary_robustness'] = successful_tests / total_boundary_tests if total_boundary_tests > 0 else 0
        return results
    
    def verify_logical_equivalence(self, equivalent_formulations: List[Tuple[str, str]]) -> Dict[str, Any]:
        """
        验证逻辑上等价的不同表述是否产生相同结果
        """
        results = {
            'equivalence_accuracy': 0.0,
            'non_equivalent_pairs': [],
            'semantic_drifts': []
        }
        
        equivalent_count = 0
        total_pairs = len(equivalent_formulations)
        
        for form1, form2 in equivalent_formulations:
            # 生成测试查询
            test_queries = self._generate_equivalence_test_queries(form1, form2)
            
            pair_equivalent = True
            for query1, query2 in test_queries:
                result1 = self.model.solve([query1])[0]
                result2 = self.model.solve([query2])[0]
                
                if not self._results_equivalent(result1, result2):
                    pair_equivalent = False
                    results['semantic_drifts'].append({
                        'formulation1': form1,
                        'formulation2': form2,
                        'query1': query1,
                        'query2': query2,
                        'result1': result1.result,
                        'result2': result2.result
                    })
            
            if pair_equivalent:
                equivalent_count += 1
            else:
                results['non_equivalent_pairs'].append((form1, form2))
        
        results['equivalence_accuracy'] = equivalent_count / total_pairs if total_pairs > 0 else 0
        return results
    
    def generate_comprehensive_report(self, test_suite: Dict[str, Any]) -> Dict[str, Any]:
        """
        生成综合验证报告
        """
        report = {
            'overall_logic_quality': 0.0,
            'verification_summary': {},
            'recommendations': [],
            'critical_issues': []
        }
        
        # 执行所有验证
        completeness_result = self.verify_logic_completeness(test_suite.get('completeness_cases', []))
        consistency_result = self.verify_logic_consistency(test_suite.get('rule_groups', []))
        counterfactual_result = self.verify_counterfactual_reasoning(test_suite.get('counterfactual_cases', []))
        boundary_result = self.verify_boundary_conditions(test_suite.get('predicates', []))
        equivalence_result = self.verify_logical_equivalence(test_suite.get('equivalent_formulations', []))
        
        # 汇总结果
        report['verification_summary'] = {
            'completeness': completeness_result,
            'consistency': consistency_result,
            'counterfactual_reasoning': counterfactual_result,
            'boundary_conditions': boundary_result,
            'logical_equivalence': equivalence_result
        }
        
        # 计算总体质量分数
        scores = [
            completeness_result['coverage_rate'],
            consistency_result['consistency_score'],
            counterfactual_result['counterfactual_accuracy'],
            boundary_result['boundary_robustness'],
            equivalence_result['equivalence_accuracy']
        ]
        report['overall_logic_quality'] = np.mean(scores)
        
        # 生成建议
        report['recommendations'] = self._generate_recommendations(report['verification_summary'])
        
        # 识别关键问题
        report['critical_issues'] = self._identify_critical_issues(report['verification_summary'])
        
        return report
    
    def _verify_logic_path(self, result, expected_path):
        """验证逻辑推理路径"""
        # 这里需要根据具体的逻辑表示来实现
        # 可能需要访问proof对象来分析推理路径
        return True  # 简化实现
    
    def _detect_rule_conflicts(self, rule_groups):
        """检测规则冲突"""
        conflicts = []
        # 实现规则冲突检测逻辑
        return conflicts
    
    def _detect_rule_redundancies(self, rule_groups):
        """检测规则冗余"""
        redundancies = []
        # 实现规则冗余检测逻辑
        return redundancies
    
    def _detect_circular_dependencies(self, rule_groups):
        """检测循环依赖"""
        circular_deps = []
        # 实现循环依赖检测逻辑
        return circular_deps
    
    def _generate_counterfactuals(self, base_case):
        """生成反事实测试用例"""
        counterfactuals = []
        # 实现反事实生成逻辑
        return counterfactuals
    
    def _analyze_result_change(self, base_result, cf_result):
        """分析结果变化"""
        change = {}
        # 实现结果变化分析逻辑
        return change
    
    def _matches_expected_change(self, actual_change, expected_change):
        """检查变化是否符合预期"""
        return True  # 简化实现
    
    def _generate_boundary_cases(self, predicate):
        """生成边界条件测试用例"""
        boundary_cases = []
        # 实现边界用例生成逻辑
        return boundary_cases
    
    def _generate_equivalence_test_queries(self, form1, form2):
        """生成等价性测试查询"""
        test_queries = []
        # 实现等价性测试查询生成逻辑
        return test_queries
    
    def _results_equivalent(self, result1, result2):
        """检查两个结果是否等价"""
        # 比较结果的概率分布
        if len(result1.result) != len(result2.result):
            return False
        
        for key in result1.result:
            if key not in result2.result:
                return False
            if abs(float(result1.result[key]) - float(result2.result[key])) > 1e-6:
                return False
        
        return True
    
    def _generate_recommendations(self, verification_summary):
        """生成改进建议"""
        recommendations = []
        
        # 基于验证结果生成具体建议
        completeness = verification_summary['completeness']
        if completeness['coverage_rate'] < 0.8:
            recommendations.append("逻辑规则覆盖率较低，建议添加更多规则处理边缘情况")
        
        consistency = verification_summary['consistency']
        if consistency['consistency_score'] < 0.9:
            recommendations.append("发现逻辑规则一致性问题，建议检查并解决规则冲突")
        
        return recommendations
    
    def _identify_critical_issues(self, verification_summary):
        """识别关键问题"""
        critical_issues = []
        
        # 识别可能导致系统故障的关键问题
        for verification_type, results in verification_summary.items():
            if verification_type == 'consistency' and results['consistency_score'] < 0.7:
                critical_issues.append({
                    'type': 'consistency',
                    'severity': 'high',
                    'description': '逻辑规则存在严重一致性问题',
                    'conflicts': results['conflicts']
                })
        
        return critical_issues


# 使用示例
def create_verification_test_suite():
    """创建测试套件示例"""
    test_suite = {
        'completeness_cases': [
            {
                'query': Query(Term('addition', Constant(1), Constant(2), Var('X'))),
                'expected_logic_path': ['digit_recognition', 'arithmetic_operation']
            }
        ],
        'rule_groups': [
            ['addition_rule_1', 'addition_rule_2'],
            ['digit_rule_1', 'digit_rule_2']
        ],
        'counterfactual_cases': [
            {
                'query': Query(Term('game', Term('cards'), Term('win'))),
                'expected_change': {'probability_increase': True}
            }
        ],
        'predicates': ['addition', 'digit', 'game'],
        'equivalent_formulations': [
            ('addition(X,Y,Z) :- digit(X,X2), digit(Y,Y2), Z is X2+Y2.',
             'addition(A,B,C) :- digit(A,A1), digit(B,B1), C is A1+B1.')
        ]
    }
    return test_suite

# 主验证流程
def run_logic_verification(model: Model):
    """运行逻辑验证的主流程"""
    verifier = LogicVerificationFramework(model)
    test_suite = create_verification_test_suite()
    
    # 生成综合报告
    report = verifier.generate_comprehensive_report(test_suite)
    
    print(f"逻辑质量总分: {report['overall_logic_quality']:.2f}")
    print(f"发现 {len(report['critical_issues'])} 个关键问题")
    print(f"建议: {len(report['recommendations'])} 条改进建议")
    
    return report

In [3]:
run_logic_verification("""nn(mnist_net,[X],Y,[0,1,2,3,4,5,6,7,8,9]) :: digit(X,Y).
addition(X,Y,Z) :- digit(X,X2), digit(Y,Y2), Z is X2+Y2.""")

AttributeError: 'str' object has no attribute 'solve'